### Práctica sobre Desarrollo de aplicaciones web con Bottle

Se desea ampliar la práctica 5 sobre las plantas medicinales creando una aplicación web que actúe a modo de capa de presentación. Para ello se va a utilizar Bottle.

Se pide crear una aplicación web que tenga una página principal que mostrará un conjunto enlaces que representan los servicios que ofrece la aplicación[1 punto]:

   * __Servicio 1__: Mostrar información sobre una planta seleccionada. Cuando el usuario pulsa sobre el servicio 1 se le mostrará un formulario en el que dispondrá varios desplegables donde podrá seleccionar la planta. Un seleccionable para elegir el grupo , y otro para seleccionar la planta del grupo que ha elegido.  Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página que mostrará la descripción de la planta. En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]
   
   
   * __Servicio 2__:Buscar planta por palabra clave. Si elige esta opción se le pedirá al usuario que introduzca un conjunto de palabras que se utilizarán para realizar una búsqueda sobre el texto de las descripciones asociadas a las plantas que aparecen en cada página. A continuación, si ha introducido más de una palabra, se le preguntará si quiere realizar una búsqueda de tipo "AND" o una búsqueda de tipo "OR", es decir si busca plantas donde aparece todas las palabras introducidas o si busca plantas donde aparecen alguna de las palabras introducidas. Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página con un listado de todas las plantas junto a las descripciones de las mismas que cumplen las condiciones de búsqueda.  En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]

   * __Servicio 3__:Buscar plantas por enfermedades. Si elige esta opción se le mostrará un listado chequeable donde seleccionará una o más enfermedades. Cuando pulse sobre un botón de tipo "Enviar", se le mostrará una nueva página con todas las plantas que pueden venir bien para esa enfermedad. Para ello se proporciona junto a la práctica un csv con nombres de enfermedades que servirá de entrada para generar el listado de enfermedades.En la página del formulario como en la del resultado habrá un enlace para volver a la página inicial[3 puntos]

## Normas de entrega

* Fecha tope de entrega: 31/10/2019
* La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre DesarrolloWeb_GrupoX donde X será el número de grupo correspondiente.

In [1]:
from bottle import route, default_app, template, run, static_file, error, Bottle,request
from lxml import etree
import urllib
from bs4 import BeautifulSoup
from string import digits
import re
import csv

def listaPlantas(indice):
        if indice == "grupoAB":
            html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(A-B)").read()
        elif indice == "grupoC":
            html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(C)").read()
        elif indice == "grupoDG":
            html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(D-G)").read()
        elif indice == "grupoHM":
            html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(H-M)").read()
        elif indice == "grupoNZ":
            html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(N-Z)").read()
            
        listaNombrePlantas=[]
        soup=BeautifulSoup(html, 'html.parser')
        etiquetaActual=soup.h3
        nombre=etiquetaActual.find('span',{"class":"mw-headline"}).string
        listaNombrePlantas.append(nombre)

        seguir=True
        while seguir:
            etiquetaActual=etiquetaActual.next_sibling
            if etiquetaActual.name == 'h3':
                nombre=etiquetaActual.find('span',{"class":"mw-headline"}).string
                listaNombrePlantas.append(nombre)
            if etiquetaActual.name == 'h2':
                seguir=False
                    
        return listaNombrePlantas
    
def plantaIndice(nombrePlanta, grupo):
    dicPlantas=crearDiccionario(grupo)
    resultado={nombrePlanta:dicPlantas[nombrePlanta]}
    return resultado

def crearDiccionario(indice):
    dicPlantas={}
    html=""
    if indice == "grupoAB":
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(A-B)").read()
        aniadirPlantasADicc(html, dicPlantas)
    elif indice == "grupoC":
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(C)").read()
        aniadirPlantasADicc(html, dicPlantas)
    elif indice == "grupoDG":
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(D-G)").read()
        aniadirPlantasADicc(html, dicPlantas)
    elif indice == "grupoHM":
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(H-M)").read()
        aniadirPlantasADicc(html, dicPlantas)
    elif indice == "grupoNZ":
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(N-Z)").read()
        aniadirPlantasADicc(html, dicPlantas)
    elif indice == "all":
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(A-B)").read()
        aniadirPlantasADicc(html, dicPlantas)
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(C)").read()
        aniadirPlantasADicc(html, dicPlantas)
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(D-G)").read()
        aniadirPlantasADicc(html, dicPlantas)
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(H-M)").read()
        aniadirPlantasADicc(html, dicPlantas)
        html=urllib.request.urlopen("https://es.wikipedia.org/wiki/Anexo:Plantas_medicinales_(N-Z)").read()
        aniadirPlantasADicc(html, dicPlantas)
        
    return dicPlantas
    
def aniadirPlantasADicc(html, dicPlantas):
    soup=BeautifulSoup(html, 'html.parser')
    etiquetaActual=soup.h3
    nombre=etiquetaActual.find('span',{"class":"mw-headline"}).string

    dicPlantas[nombre] = []
    
    etiquetaNombreCientifico=etiquetaActual.next_sibling
    isCorrectNext_sibling=False
    while not isCorrectNext_sibling:
        if etiquetaNombreCientifico.name == 'dl':
            isCorrectNext_sibling=True
        else:
            etiquetaNombreCientifico=etiquetaNombreCientifico.next_sibling

    dt=etiquetaNombreCientifico.find('dt')
    if not dt==None:
        nombreCientificoRAW=dt.find_all(text=True)
    nombreCientifico=""
    for parte in nombreCientificoRAW:
        nombreCientifico=nombreCientifico+parte
            
    dicPlantas[nombre].append(nombreCientifico)
    
    descripcion=""
    etiquetaActual=etiquetaActual.next_sibling
    isCorrectNext_sibling=False
    while not isCorrectNext_sibling:
        if etiquetaActual.name == 'dl':
            isCorrectNext_sibling=True
        else:
            etiquetaActual=etiquetaActual.next_sibling

    seguir=True
    while seguir:
        if etiquetaActual.name == 'dl':
            for elemento in etiquetaActual.find_all('dd'):
                for parrafo in elemento.find_all(text=True):
                    if ((len(parrafo) <= 1) and (parrafo == '.' or parrafo == ',' or parrafo == ';' or parrafo == ':' or parrafo == "'" or parrafo == '"' or parrafo == '?' or parrafo == '!' or parrafo == '-')) or ((len(parrafo) > 1) and not parrafo.isdigit()):
                        descripcion=descripcion+str(parrafo)
            etiquetaActual=etiquetaActual.next_sibling
        elif etiquetaActual.name == 'h3':
            seguir=False
        elif etiquetaActual.name == 'h2':
            seguir=False
        else:
            etiquetaActual=etiquetaActual.next_sibling
            
    dicPlantas[nombre].append(descripcion)
    
    seguir=True
    while seguir:
        if etiquetaActual.name == 'h3':
            nombre=etiquetaActual.find('span',{"class":"mw-headline"}).string
            dicPlantas[nombre] = []
            etiquetaNombreCientifico=etiquetaActual.next_sibling
            isCorrectNext_sibling=False
            while not isCorrectNext_sibling:
                if etiquetaNombreCientifico.name == 'dl':
                    isCorrectNext_sibling=True
                else:
                    etiquetaNombreCientifico=etiquetaNombreCientifico.next_sibling
            
            dt=etiquetaNombreCientifico.find('dt')
            if not dt==None:
                nombreCientificoRAW=dt.find_all(text=True)
            nombreCientifico=""
            for parte in nombreCientificoRAW:
                nombreCientifico=nombreCientifico+parte
            dicPlantas[nombre].append(nombreCientifico)

            descripcion=""
            etiquetaActual=etiquetaActual.next_sibling
            isCorrectNext_sibling=False
            while not isCorrectNext_sibling:
                if etiquetaActual.name == 'dl':
                    isCorrectNext_sibling=True
                else:
                    etiquetaActual=etiquetaActual.next_sibling

            seguirDes=True
            while seguirDes:
                if etiquetaActual.name == 'dl':
                    for elemento in etiquetaActual.find_all('dd'):
                        for parrafo in elemento.find_all(text=True):
                            if ((len(parrafo) <= 1) and (parrafo == '.' or parrafo == ',' or parrafo == ';' or parrafo == ':' or parrafo == "'" or parrafo == '"' or parrafo == '?' or parrafo == '!' or parrafo == '-')) or ((len(parrafo) > 1) and not parrafo.isdigit()):
                                descripcion=descripcion+str(parrafo)
                    etiquetaActual=etiquetaActual.next_sibling
                elif etiquetaActual.name == 'h3':
                    seguirDes=False
                elif etiquetaActual.name == 'h2':
                    seguirDes=False
                else:
                    etiquetaActual=etiquetaActual.next_sibling
            dicPlantas[nombre].append(descripcion)
            
        elif etiquetaActual.name == 'h2':
            seguir=False
        else:
            etiquetaActual=etiquetaActual.next_sibling
            
def palabraClave(textoBusqueda, tipoBusqueda):
    dicPlantas=crearDiccionario("all")
    listaPalabrasBusqueda=textoBusqueda.split()
    resultado=buscarPlanta(dicPlantas, listaPalabrasBusqueda, tipoBusqueda)
    
    return resultado

def buscarPlanta(dicPlantas, listaPalabras, tipoBusqueda):
    resultado = {}
    #BUSQUEDA POR OR
    if tipoBusqueda == "or":
        for nombrePlanta, listaInfo in dicPlantas.items():
            mostrar = False
            i = 0
            listaPalabrasDescripcion=listaInfo[1].lower().split()
            while mostrar == False and i < len(listaPalabras):
                j=0
                while mostrar == False and j < len(listaPalabrasDescripcion):
                    if listaPalabrasDescripcion[j] == listaPalabras[i].lower():
                        mostrar = True
                    j += 1
                i += 1
            if mostrar == True:
                resultado[nombrePlanta] = listaInfo
     
    #BUSQUEDA POR AND
    elif tipoBusqueda == "and":
        for nombrePlanta, listaInfo in dicPlantas.items():
            mostrar = True
            i = 0
            listaPalabrasDescripcion=listaInfo[1].lower().split()
            while mostrar == True and i < len(listaPalabras):
                j=0
                encontrado = False
                while encontrado == False and j < len(listaPalabrasDescripcion):
                    if listaPalabrasDescripcion[j] == listaPalabras[i].lower():
                        encontrado = True
                    j += 1
                mostrar = encontrado
                i += 1
            if mostrar == True:
                resultado[nombrePlanta] = listaInfo
                
    return resultado
            
def abrirEnfermedades():
    csvarchivo = open("Enfermedades.csv", encoding = "latin-1")
    entrada = csv.reader(csvarchivo, delimiter="\n")
    listaEnfermedades = []
    for fila in entrada:
        if entrada.line_num == 1:
            continue
        if fila[0] not in listaEnfermedades:
            listaEnfermedades.append(fila[0])
    csvarchivo.close()
    return listaEnfermedades

def busquedaEnfermedad(listaEnfermedades):
    resultado = {}
    dicPlantas = crearDiccionario("all")
    for nombrePlanta, listaInfo in dicPlantas.items():
        mostrar = False
        i=0
        while mostrar == False and i < len(listaEnfermedades):
            if not listaInfo[1].lower().find(listaEnfermedades[i].lower()) == -1:
                mostrar = True
                
            i += 1
        if mostrar == True:
            resultado[nombrePlanta] = listaInfo
    return resultado

@route('/')
def index():
    return template("index.tpl")

@route('/indice')
def indice():
    return template("indice.tpl")

@route('/indice' ,method='POST')
def do_indice():
    opcion = request.forms.get('grupo')
    lista=listaPlantas(opcion)
    return template("planta.tpl", plantas=lista, grupo=opcion)

@route('/planta' ,method='POST')
def do_planta():
    nombrePlanta = request.forms.get('plantas')
    grupo = request.forms.get('grupo')
    planta = plantaIndice(nombrePlanta, grupo)
    return template("resultado.tpl", plantas=planta)

@route('/palabras')
def palabras():
    return template("palabras.tpl")

@route('/palabras' ,method='POST')
def do_palabras():
    textoBusqueda = request.forms.get('texto')
    if len(textoBusqueda.split()) > 1:
        return template("tipoBusqueda.tpl", texto=textoBusqueda)
    else:
        opcion = "or"
        textoBusqueda = textoBusqueda.strip()
        plantas=palabraClave(textoBusqueda, opcion)
        return template("resultado.tpl", plantas=plantas)

@route('/tipoBusqueda' ,method='POST')
def do_tipoBusqueda():
    opcion = request.forms.get('opcion')
    textoBusqueda = request.forms.get('texto')
    textoBusqueda = textoBusqueda.strip()
    plantas=palabraClave(textoBusqueda, opcion)
    return template("resultado.tpl", plantas=plantas)

@route('/enfermedades')
def enfermedades():
    enfermedades=abrirEnfermedades()
    return template("enfermedades.tpl", enfermedades=enfermedades)

@route('/enfermedades' ,method='POST')
def do_enfermedades():
    listaEnfermedades=request.forms.getall('enfermedad')
    plantas = busquedaEnfermedad(listaEnfermedades)
    return template("resultado.tpl", plantas=plantas)

@error(404)
def error404(error):
    return template("OjmeKP.tpl")

@route('/static/<filepath:path>')
def server_static(filepath):
    return static_file(filepath, root='static')

ModuleNotFoundError: ignored

In [0]:
run(host='0.0.0.0', port=8080)